In [ ]:
import altair as alt
import pandas as pd


# FBI Data

In [ ]:
df = pd.read_csv('../data/hate_crime_fbi.csv')

alt.data_transformers.disable_max_rows()

df.columns

In [ ]:
# Filtering by keywords
black_df = df[df['bias_desc'].str.contains('Anti-Black', case=False, na=False)].copy()
hispanic_df = df[df['bias_desc'].str.contains('Anti-Hispanic', case=False, na=False)].copy()
asian_df = df[df['bias_desc'].str.contains('Anti-Asian', case=False, na=False)].copy()

# Assigning labels to each category
black_df['group'] = 'Anti-Black'
hispanic_df['group'] = 'Anti-Hispanic'
asian_df['group'] = 'Anti-Asian'

# Combining the filtered DataFrames
combined_df = pd.concat([black_df, hispanic_df, asian_df])

# Aggregating the number of incidents per year
counts_by_year = (
    combined_df.groupby(['data_year', 'group'])
    .size()
    .reset_index(name='count')
)

counts_by_year.to_csv("counts_by_year.csv", index=False)

chart = alt.Chart(counts_by_year).mark_line(point=True).encode(
    x=alt.X('data_year:O', title='Year'),
    y=alt.Y('count:Q', title='Number of Incidents'),
    color=alt.Color('group:N', title='Bias Type')
).properties(
    width=600,
    height=400,
    title='Hate Crime Incidents by Bias Type and Year'
)

chart

In [ ]:
# Filter by keywords
black_df = df[df['bias_desc'].str.contains('Anti-Black', case=False, na=False)].copy()
hispanic_df = df[df['bias_desc'].str.contains('Anti-Hispanic', case=False, na=False)].copy()
asian_df = df[df['bias_desc'].str.contains('Anti-Asian', case=False, na=False)].copy()

# Assign labels to each category
black_df['group'] = 'Anti-Black'
hispanic_df['group'] = 'Anti-Hispanic'
asian_df['group'] = 'Anti-Asian'

# Combine the filtered DataFrames
combined_df = pd.concat([black_df, hispanic_df, asian_df])

# Aggregate the number of incidents per year
counts_by_year = (
    combined_df.groupby(['data_year', 'group'])
    .size()
    .reset_index(name='count')
)

# Population by race/ethnicity (based on 2020 Census, fixed)
population_dict = {
    'Anti-Black': 41288572,
    'Anti-Hispanic': 61755866,
    'Anti-Asian': 19112979
}

# Map population and calculate incidents per 100,000 people
counts_by_year['population'] = counts_by_year['group'].map(population_dict)
counts_by_year['per_100k'] = counts_by_year['count'] / counts_by_year['population'] * 100000

# Draw the chart with Altair (per capita basis)
chart = alt.Chart(counts_by_year).mark_line(point=True).encode(
    x=alt.X('data_year:O', title='Year'),
    y=alt.Y('per_100k:Q', title='Incidents per 100,000 People'),
    color=alt.Color('group:N', title='Bias Type'),
    tooltip=['data_year', 'group', 'per_100k']
).properties(
    width=600,
    height=400,
    title='Hate Crime Incidents per 100,000 People by Bias Type and Year'
)

counts_by_year.to_csv("counts_per_100k.csv", index=False)

chart

In [ ]:
# Columns to compare
columns_to_compare = ['victim_types', 'location_name', 'offender_race']

# Dictionary to store the results
results = {}

# Filter by each bias group
groups = {
    'Anti-Black': df[df['bias_desc'].str.contains('Anti-Black', case=False, na=False)].copy(),
    'Anti-Hispanic': df[df['bias_desc'].str.contains('Anti-Hispanic', case=False, na=False)].copy(),
    'Anti-Asian': df[df['bias_desc'].str.contains('Anti-Asian', case=False, na=False)].copy()
}

# Aggregate for each column
for col in columns_to_compare:
    frames = []
    for group_name, group_df in groups.items():
        # Calculate percentages
        count = group_df[col].value_counts(dropna=False, normalize=True) * 100
        count_df = count.rename(f'{group_name} (%)').reset_index()
        count_df.columns = [col, f'{group_name} (%)']
        frames.append(count_df.set_index(col))
    
    # Join horizontally
    comparison_df = pd.concat(frames, axis=1).fillna(0).reset_index()
    results[col] = comparison_df

# Display results by category (uncomment as needed)

print("=== Comparison by victim_types ===")
print(results['victim_types'].to_string(index=False))
print("\n=== Comparison by location_name ===")
print(results['location_name'].to_string(index=False))
print("\n=== Comparison by offender_race ===")
print(results['offender_race'].to_string(index=False))

# Examine

In [ ]:
df = pd.read_csv('2015-2023_03.csv')


In [ ]:
# Anti-Black=12, Anti-Asian=14, Anti-Hispanic=32
target_codes = [12, 14, 32]

race_df = df[df['V20201'].isin(target_codes)]

In [ ]:
import pandas as pd
import altair as alt

# Target bias motivation codes and their labels
target_codes = {
    12: 'Anti-Black',
    14: 'Anti-Asian',
    32: 'Anti-Hispanic'
}

# Define a parser that can handle multiple date formats
def parse_date(date_str):
    for fmt in ('%Y%m%d', '%d-%b-%Y', '%Y-%m-%d'):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except (ValueError, TypeError):
            continue
    return pd.NaT

# Clean INCDATE in the entire race_df
race_df['INCDATE'] = race_df['INCDATE'].apply(parse_date)

# Remove rows with unparseable dates
race_df = race_df.dropna(subset=['INCDATE'])

# Extract year from the cleaned date
race_df['YEAR'] = race_df['INCDATE'].dt.year

# Filter rows where V20201 matches any of the target codes
race_df_filtered = race_df[race_df['V20201'].isin(target_codes.keys())].copy()

# Map bias code to label
race_df_filtered['label'] = race_df_filtered['V20201'].map(target_codes)

# Aggregate number of incidents by year and bias label
yearly_counts = (
    race_df_filtered
    .groupby(['YEAR', 'label'])
    .size()
    .reset_index(name='count')
)

# Create Altair line chart
chart = alt.Chart(yearly_counts).mark_line(point=True).encode(
    x=alt.X('YEAR:O', title='Year'),
    y=alt.Y('count:Q', title='Number of Incidents'),
    color=alt.Color('label:N', title='Bias Motivation'),
    tooltip=['YEAR', 'label', 'count']
).properties(
    width=600,
    height=400,
    title='Hate Crime Incidents by Year and Bias Motivation (V20201 only)'
)

chart

## Seasonality

In [ ]:
import pandas as pd
import altair as alt

# Ensure INCDATE is already parsed to datetime in race_df
race_df['MONTH'] = race_df['INCDATE'].dt.month

# Target bias motivation codes and their labels
target_codes = {
    12: 'Anti-Black',
    14: 'Anti-Asian',
    32: 'Anti-Hispanic'
}

# Filter for selected bias motivations
race_df_filtered = race_df[race_df['V20201'].isin(target_codes.keys())].copy()
race_df_filtered['label'] = race_df_filtered['V20201'].map(target_codes)

# Count incidents by month and label
monthly_counts = (
    race_df_filtered
    .groupby(['label', 'MONTH'])
    .size()
    .reset_index(name='count')
)

# Calculate percentage by group (label)
monthly_counts['percentage'] = (
    monthly_counts
    .groupby('label')['count']
    .transform(lambda x: x / x.sum() * 100)
)

# Make sure month is categorical for correct ordering
monthly_counts['MONTH'] = monthly_counts['MONTH'].astype(int).astype(str)

# Altair chart: percentage of incidents by month
chart = alt.Chart(monthly_counts).mark_line(point=True).encode(
    x=alt.X('MONTH:O', title='Month'),
    y=alt.Y('percentage:Q', title='Share of Incidents (%)'),
    color=alt.Color('label:N', title='Bias Motivation'),
    tooltip=['label', 'MONTH', 'percentage']
).properties(
    width=600,
    height=400,
    title='Monthly Distribution of Hate Crimes by Bias Motivation (as % of Total)'
)

chart

### UCR OFFENSE CODE

In [ ]:
# Cross-tabulation (frequency)
offense_bias_table = pd.crosstab(race_df['V20201'], race_df['V20061'])

# Display the result
print(offense_bias_table)

# Cross-tabulation (percentage per V20201)
offense_bias_pct = pd.crosstab(race_df['V20201'], race_df['V20061'], normalize='index') * 100

# Display rounded to two decimal places
print(offense_bias_pct.round(2))

import altair as alt

# Convert data to long format
offense_bias_df = offense_bias_pct.reset_index().melt(
    id_vars='V20201', 
    var_name='V20061', 
    value_name='percentage'
)

# Draw a heatmap with Altair
heatmap = alt.Chart(offense_bias_df).mark_rect().encode(
    x=alt.X('V20061:N', title='UCR Offense Code (V20061)'),
    y=alt.Y('V20201:N', title='Bias Motivation (V20201)'),
    color=alt.Color('percentage:Q', title='Percentage (%)', scale=alt.Scale(scheme='blues')),
    tooltip=['V20201', 'V20061', 'percentage']
).properties(
    title='Bias Motivation by UCR Offense Code',
    width=600,
    height=300
)

heatmap

### RELATIONSHIP VIC TO OFF

In [ ]:
# Cross-tabulation (showing percentages)
cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V40321'], normalize='index') * 100

# Round to two decimal places
print(cross_table_pct.round(2))

In [ ]:
race_df['V40321_filled'] = race_df['V40321'].astype(str).replace({'nan': 'Unknown'})

cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V40321_filled'], normalize='index') * 100

print(cross_table_pct.round(2))

In [ ]:
# Map V20201 bias motivation to target group labels
bias_mapping = {
    12: 'Anti-Black',
    14: 'Anti-Asian',
    32: 'Anti-Hispanic'
}
race_df['bias_group'] = race_df['V20201'].map(bias_mapping)

# Function to map victim-offender relationship to 3 categories
def classify_relationship(val):
    try:
        val = int(val)
        if val == 25:
            return 'Stranger'
        elif val in list(range(1, 25+1)) + [26]:
            return 'Acquaintance'
        elif val == 27:
            return 'Unknown'
        else:
            return 'Unknown'
    except:
        return 'Unknown'

# Apply the relationship classification
race_df['relationship_category'] = race_df['V40321'].apply(classify_relationship)

# Crosstab by bias group and relationship category, showing row-wise percentages
cross_table = pd.crosstab(race_df['bias_group'], race_df['relationship_category'], normalize='index') * 100

# Print rounded table
print(cross_table.round(2))

In [ ]:
# Step 1: Crosstab with original V40321 values (normalized to row percentages)
raw_pct = pd.crosstab(race_df['V20201'], race_df['V40321'], normalize='index') * 100

# Step 2: Define grouping of original V40321 codes
group_map = {
    'Stranger': [25],
    'Unknown': [0, 27, -9, -8, -7, -6, -5],
    'Acquaintance': []  # we will fill this below
}

# Step 3: Get actual codes that exist in the data
actual_codes = set(raw_pct.columns)

# Step 4: Automatically assign all remaining codes to 'Acquaintance'
used_codes = set(group_map['Stranger'] + group_map['Unknown'])
group_map['Acquaintance'] = list(actual_codes - used_codes)

# Step 5: Aggregate using original row-normalized percentages
simplified_pct = pd.DataFrame(index=raw_pct.index)

for group, codes in group_map.items():
    # Only sum columns that actually exist in raw_pct
    valid_codes = [code for code in codes if code in raw_pct.columns]
    if valid_codes:
        simplified_pct[group] = raw_pct[valid_codes].sum(axis=1)
    else:
        simplified_pct[group] = 0  # If no valid codes, fill with 0s

# Step 6: Rename race codes to readable labels
race_labels = {12: 'Anti-Asian', 14: 'Anti-Black', 32: 'Anti-Hispanic'}
simplified_pct.index = simplified_pct.index.map(race_labels)

# Step 7: Display
print(simplified_pct.round(2))

In [ ]:
import pandas as pd

# Step 1: Map V20201 to bias group labels
bias_mapping = {
    12: 'Anti-Black',
    14: 'Anti-Asian',
    32: 'Anti-Hispanic'
}
race_df['bias_group'] = race_df['V20201'].map(bias_mapping)

# Step 2: Map V40321 to relationship category
def classify_relationship(val):
    try:
        val = int(val)
        if val == 25:
            return 'Stranger'
        elif val in list(range(1, 26)) + [26]:
            return 'Acquaintance'
        elif val == 27:
            return 'Unknown'
        else:
            return 'Unknown'
    except:
        return 'Unknown'

race_df['relationship_category'] = race_df['V40321'].apply(classify_relationship)

# Step 3: Crosstab with row-wise normalization (percentages)
cross_table = pd.crosstab(race_df['bias_group'], race_df['relationship_category'], normalize='index') * 100

# Step 4: Save to CSV for D3 use
cross_table_csv = cross_table.reset_index()
cross_table_csv.to_csv("relationship_distribution.csv", index=False)

## Victim

In [ ]:
# Cross-tabulation (showing percentages)
cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V40221'], normalize='index') * 100

# Round to two decimal places
print(cross_table_pct.round(2))

### Victim Sex

In [ ]:
race_df['V40191'] = pd.to_numeric(race_df['V40191'], errors='coerce')

cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V40191'], normalize='index') * 100

print(cross_table_pct.round(2))

### Victim Race

In [ ]:
# Cross-tabulation (showing percentages)
cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V40201'], normalize='index') * 100

# Round to two decimal places
print(cross_table_pct.round(2))

### Victim Age

In [ ]:

race_df['V40181'] = pd.to_numeric(race_df['V40181'], errors='coerce')

race_df_clean = race_df.dropna(subset=['V40181'])

race_df_clean['age_group'] = pd.cut(
    race_df_clean['V40181'],
    bins=range(0, 101, 10),
    right=False,
    labels=['0–9', '10–19', '20–29', '30–39', '40–49',
            '50–59', '60–69', '70–79', '80–89', '90–99']
)

cross_table_pct = pd.crosstab(
    race_df_clean['V20201'],
    race_df_clean['age_group'],
    normalize='index'
) * 100

race_labels = {
    12: 'Anti-Black',
    14: 'Anti-Asian',
    32: 'Anti-Hispanic'
}
cross_table_pct.index = cross_table_pct.index.map(race_labels)


print(cross_table_pct.round(2))

In [ ]:
# Extract Data
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde


race_df['V40181'] = pd.to_numeric(race_df['V40181'], errors='coerce')
df = race_df.dropna(subset=['V40181'])
race_labels = {12: 'Anti-Black', 14: 'Anti-Asian', 32: 'Anti-Hispanic'}
df = df[df['V20201'].isin(race_labels)].copy()
df['race_label'] = df['V20201'].map(race_labels)

output = []

x_grid = np.linspace(0, 100, 200)

for label in df['race_label'].unique():
    values = df[df['race_label'] == label]['V40181'].dropna()
    kde = gaussian_kde(values)
    densities = kde(x_grid)
    for x, y in zip(x_grid, densities):
        output.append({'race_label': label, 'age': x, 'density': y})

output_df = pd.DataFrame(output)
output_df.to_csv('age_density_by_race.csv', index=False)

In [ ]:
import altair as alt

chart = alt.Chart(df).transform_density(
    'V40181',
    as_=['age', 'density'],
    groupby=['race_label']
).mark_line(opacity=0.7, strokeWidth=2).encode(
    x=alt.X('age:Q', title='Age'),
    y=alt.Y('density:Q', title='Density'),
    color=alt.Color('race_label:N', title='Bias Motivation')
).properties(
    width=600,
    height=300,
    title='Age Density Comparison by Bias Motivation'
)

chart

In [ ]:
import altair as alt
import pandas as pd

# Convert age to numeric
race_df['V40181'] = pd.to_numeric(race_df['V40181'], errors='coerce')

# Drop rows with missing age and filter for target racial bias groups
df = race_df.dropna(subset=['V40181'])
race_labels = {12: 'Anti-Black', 14: 'Anti-Asian', 32: 'Anti-Hispanic'}
df = df[df['V20201'].isin(race_labels.keys())].copy()
df['race_label'] = df['V20201'].map(race_labels)

# Restrict age range to 0–99 and convert to integer
df = df[(df['V40181'] >= 0) & (df['V40181'] < 100)]
df['age'] = df['V40181'].astype(int)

# Create a cross table of counts per race and age
counts = (
    df.groupby(['race_label', 'age'])
    .size()
    .reset_index(name='count')
)

# Convert to percentage within each race group
counts['percentage'] = (
    counts.groupby('race_label')['count']
    .transform(lambda x: x / x.sum() * 100)
)

# Draw Altair heatmap
heatmap = alt.Chart(counts).mark_rect().encode(
    x=alt.X('age:O', title='Age'),
    y=alt.Y('race_label:N', title='Bias Motivation'),
    color=alt.Color('percentage:Q', title='Share (%)', scale=alt.Scale(scheme='reds')),
    tooltip=['race_label', 'age', 'percentage']
).properties(
    width=600,
    height=150,
    title='Age Distribution of Victims by Bias Motivation (Heatmap)'
)

heatmap

## Sex x Age of Victim

In [ ]:
import pandas as pd
import altair as alt

bias_map = {12: "Anti-Black", 14: "Anti-Asian", 32: "Anti-Hispanic"}
sex_map = {0: "Female", 1: "Male"}

df = race_df[
    race_df["V20201"].isin(bias_map.keys()) &
    race_df["V40191"].isin(sex_map.keys())
].copy()

df["bias_group"] = df["V20201"].map(bias_map)
df["sex"] = df["V40191"].map(sex_map)
df["age"] = pd.to_numeric(df["V40181"], errors="coerce")
df["age_range"] = pd.cut(df["age"], bins=range(0, 100, 5), right=False).astype(str)
age_range_order = sorted(df["age_range"].unique().tolist())

grouped = (
    df.groupby(["bias_group", "sex", "age_range"])
    .size()
    .reset_index(name="count")
)

grouped["percent"] = grouped.groupby("bias_group")["count"].transform(lambda x: x / x.sum() * 100)


chart = alt.Chart(grouped).mark_rect().encode(
    x=alt.X("age_range:N", title="Age Range", sort=age_range_order),
    y=alt.Y("sex:N", title="Sex"),
    color=alt.Color("percent:Q", title="Percentage", scale=alt.Scale(scheme="blues"))
).properties(
    width=400,
    height=80
)

final_chart = chart.facet(
    row=alt.Row("bias_group:N", title=None, sort=["Anti-Asian", "Anti-Black", "Anti-Hispanic"])
).properties(
    title="Age and Sex Distribution by Bias Motivation (Percentage)"
)

final_chart

### Location Type

In [ ]:
# Cross-tabulation (showing percentages)
cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V20111'], normalize='index') * 100

# Round to two decimal places
print(cross_table_pct.round(2))

In [ ]:
# Dictionary for location category classification
location_map = {
    20: 'Residence', 54: 'Residence',
    3: 'Commercial', 7: 'Commercial', 8: 'Commercial',
    12: 'Commercial', 14: 'Commercial', 17: 'Commercial',
    21: 'Commercial', 23: 'Commercial', 24: 'Commercial',
    55: 'Commercial',
    1: 'Road', 13: 'Road', 18: 'Road',
    39: 'Road', 40: 'Road', 51: 'Road',
    4: 'School', 11: 'School', 22: 'School',
    50: 'School', 53: 'School', 57: 'School',
    5: 'Other', 6: 'Other', 15: 'Other', 25: 'Other',
    37: 'Other', 41: 'Other', 45: 'Other', 46: 'Other',
    47: 'Other', 48: 'Other', 49: 'Other', 56: 'Other',
    58: 'Other'
}

# Apply location categories to V20111
race_df.loc[:, 'LocationCategory'] = race_df['V20111'].map(location_map).fillna('Other')

# Mapping from numeric race codes to human-readable labels
race_labels = {
    12: 'Anti-Black',
    14: 'Anti-Asian',
    32: 'Anti-Hispanic'
}

# Cross-tabulation: row-wise percentage of incidents by location category
cross_table_pct_grouped = pd.crosstab(
    race_df['V20201'], race_df['LocationCategory'], normalize='index'
) * 100

# Rename index using race labels
cross_table_pct_grouped.index = cross_table_pct_grouped.index.map(race_labels)

# Add index name for correct CSV column label
cross_table_pct_grouped.index.name = 'Race'

# Reorder columns
columns_order = ['Residence', 'School', 'Commercial', 'Road', 'Other']
cross_table_pct_grouped = cross_table_pct_grouped[columns_order]

# Display table
print(cross_table_pct_grouped.round(2))

# Save to CSV
cross_table_pct_grouped.round(2).to_csv("location_distribution.csv")

## Offender

### Offender Sex

In [ ]:
# Cross-tabulation (showing percentages)
cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V50081'], normalize='index') * 100

# Round to two decimal places
print(cross_table_pct.round(2))

### Offender Race

In [ ]:
# Cross-tabulation (showing percentages)
cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V50091'], normalize='index') * 100

# Round to two decimal places
print(cross_table_pct.round(2))

### Offender Ethnicity

In [ ]:
# Cross-tabulation (showing percentages)
cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V50111'], normalize='index') * 100

# Round to two decimal places
print(cross_table_pct.round(2))

In [ ]:
# Cross-tabulation (showing percentages)
cross_table_pct = pd.crosstab(race_df['V50111'], race_df['V50091'], normalize='index') * 100

# Round to two decimal places
print(cross_table_pct.round(2))

### Offence Code

In [ ]:
# offense code → offense category mapping（例）
def categorize_offense(code):
    if pd.isna(code):
        return 'Missing'
    try:
        code = int(code)
    except:
        return 'Invalid'
    
    if 90 <= code < 100:
        return 'Homicide'
    elif 100 <= code < 130:
        return 'Sex Offense / Robbery'
    elif 130 <= code < 200:
        return 'Assault / Intimidation'
    elif 200 <= code < 240:
        return 'Property Crime'
    elif 240 <= code < 270:
        return 'Fraud / Theft'
    else:
        return 'Other'

# 新しいカテゴリ列を追加
race_df['Offense_Category'] = race_df['V20061'].apply(categorize_offense)

# クロス集計（行: V20201、列: offense category）
cross_table_pct = pd.crosstab(
    race_df['V20201'], 
    race_df['Offense_Category'], 
    normalize='index'
) * 100

# 小数点2桁で表示
print(cross_table_pct.round(2))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# クロス集計（既に集計済みと仮定）
cross_table_pct = pd.crosstab(
    race_df['V20201'],
    race_df['Offense_Category'],
    normalize='index'
) * 100

# 描画
ax = cross_table_pct.plot(kind='bar', stacked=True, figsize=(10, 6))

plt.title('Offense Category Distribution by Victim Race')
plt.xlabel('Victim Race (V20201)')
plt.ylabel('Percentage')
plt.legend(title='Offense Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

### Offender Age

In [ ]:
# Convert non-numeric values (e.g., strings) to NaN and then to numeric
race_df['V50071'] = pd.to_numeric(race_df['V50071'], errors='coerce')

# Drop rows with NaN in the age column (exclude missing age data from analysis)
race_df_clean = race_df.dropna(subset=['V50071'])

# Group age into 10-year bins
race_df_clean['age_group'] = pd.cut(
    race_df_clean['V50071'],
    bins=range(0, 101, 10),
    right=False,
    labels=['0-9','10-19','20-29','30-39','40-49',
            '50-59','60-69','70-79','80-89','90-99']
)

# Group by age_group and V20201, then reshape into long format
age_group_counts = (
    race_df_clean.groupby(['V20201', 'age_group'])
    .size()
    .reset_index(name='count')
)

# Calculate the percentage within each Bias Motivation group
age_group_counts['percentage'] = (
    age_group_counts.groupby('V20201')['count']
    .transform(lambda x: x / x.sum() * 100)
)

# Plot with Altair
import altair as alt

chart = alt.Chart(age_group_counts).mark_bar().encode(
    x=alt.X('age_group:N', title='Age Group'),
    y=alt.Y('percentage:Q', title='Percentage (%)'),
    color=alt.Color('age_group:N', legend=None),
    column=alt.Column('V20201:N', title='Bias Motivation (V20201)')
).properties(
    width=100,
    height=300,
    title='Age Distribution by Bias Motivation'
)

chart